In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import date

class AQS():
    def __init__(self, email=None, key=None):
        if email is None:
            print("If you don't have an acount please make one with the SignUp func")
        else:
            self.auth = {'email': email, 'key': key}

    def SignUp(self, email):
        payload = {"email": email}
        req = requests.get("https://aqs.epa.gov/data/api/signup", params=payload)
        if req.status_code == 200:
            print("OK!")
        else:
            print("BOO!")

    def getCounties(self, state):
        payload = {"state": state}
        req = requests.get("https://aqs.epa.gov/data/api/list/countiesByState", params= (self.auth | payload))
        return pd.DataFrame.from_dict(req.json()["Data"])

    def getParams(self):
        payload = {"pc":"CRITERIA"}
        req = requests.get("https://aqs.epa.gov/data/api/list/parametersByClass", params= (self.auth | payload))
        return pd.DataFrame.from_dict(req.json()["Data"])

    def DailySummaryState(self, param, bdate, edate, state):
        payload = {"param":param, "bdate": bdate, "edate": edate, "state": state}
        req = requests.get("https://aqs.epa.gov/data/api/dailyData/byState", params= (self.auth | payload))
        return pd.DataFrame.from_dict(req.json()["Data"])

    def DailySummaryCounty(self, param, bdate, edate, state, county):
        payload = {"param":param, "bdate": bdate, "edate": edate, "state": state, "county": county}
        req = requests.get("https://aqs.epa.gov/data/api/dailyData/byCounty", params= (self.auth | payload))
        if req.json()["Header"][0].get("status") == "No data matched your selection":
            return pd.DataFrame()
        return pd.DataFrame.from_dict(req.json()["Data"])
          
    def DailySummarySite(self, param, bdate, edate, state, county, site):
        payload = {"param":param, "bdate": bdate, "edate": edate, "state": state, "county": county, "site": site}
        req = requests.get("https://aqs.epa.gov/data/api/dailyData/bySite", params= (self.auth | payload))
        return pd.DataFrame.from_dict(req.json()["Data"])

In [2]:
epa = AQS("alb323@lehigh.edu", "indigoswift15")

data = epa.DailySummaryCounty("88101", "20220101", "20220102", "42", "095")

True

In [3]:

for standard in data["pollutant_standard"].unique():
    temp = data.loc[data["pollutant_standard"] == standard]
    for sample in temp["sample_duration"].unique():
        temp = data.loc[data["pollutant_standard"] == standard]
        temp = temp.loc[temp["sample_duration"] == sample]
        print(standard, " : ", sample, " : ", len(temp))
        #print(temp["date_local"])

KeyError: 'pollutant_standard'

In [ ]:
data["date_local"] = data["date_local"].astype("datetime64[ns]")
missing = pd.date_range(start = '2022-01-1', end = '2022-04-19' ).difference(data["date_local"])
print(missing)
print(len(missing))

KeyError: 'date_local'